TO DO

- is it possible to add captaincy bonus?
- add an optional constraint to allow only one defender from any given team (risk mitigation)?

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / 10
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 29
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 39 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,7.431697,2.477232,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,42.078915,0,0.0
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.684507,0,0.0
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.959863,0,0.0
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.5,3.6,3.8,2.4,6.8,4.071,29.530854,3,0.0
5,24.838989,2.483899,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.0,NaN,NaN,NaN,NaN,NaN,30.122843,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.655839,0,0.0
685,4.979590,2.489795,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.994789,1,0.0
686,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.609191,0,0.0


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=3]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 39 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,7.431697,2.477232,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,42.078915,0,0.0
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,29.530854,3,0.0
5,24.838989,2.483899,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.000000,NaN,NaN,NaN,NaN,NaN,30.122843,2,0.0
6,79.828000,3.628545,8,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,NaN,NaN,NaN,1.000000,3.600000,2.000000,39.781758,0,0.0
7,36.846596,2.302912,1,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,-2.000000,NaN,NaN,2.332871,NaN,NaN,NaN,26.152328,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,55.896424,2.661734,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,5.548812,4.418731,2.382085,NaN,5.048812,NaN,NaN,21.198851,0,0.0
470,55.920721,3.495045,4,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.861196,29.103007,4,0.0
475,115.475057,3.981899,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.195247,5.274923,0.328340,11.375777,4.195247,3.331484,3.961196,33.697126,3,0.0


In [5]:
optimize(df, 86, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 73.3
Expected points: 52.54371690750122


,element_type,web_name,now_cost,games played,gameweek 30 prediction
id,,,,,
1,1,Leno,45,3.0,4.307070
16,2,Tierney,51,21.0,3.886938
360,2,Doherty,48,12.0,4.032229
67,2,White,45,25.0,3.738794
196,3,Raphinha,65,27.0,4.745955
210,3,Tielemans,64,21.0,4.183875
359,3,Son,109,25.0,5.147920
681,3,Coutinho,75,9.0,4.422555
701,3,Kulusevski,62,8.0,4.676741


In [6]:
optimize(df, 86, 'adjusted points per game')

Status: Optimal
Cost: 85.80000000000001
Expected points per week: 71.98154587908314


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
129,1,Arrizabalaga,46,4.0,4.399678
135,2,Chilwell,56,6.0,6.261511
142,2,James,62,17.0,5.562972
237,2,Alexander-Arnold,85,26.0,6.508953
256,2,Cancelo,69,27.0,5.831865
233,3,Salah,133,27.0,7.815741
265,3,Foden,79,21.0,5.333030
268,3,Torres,67,4.0,6.299161
359,3,Son,109,25.0,5.693650


In [7]:
df = df.drop([1])
optimize(df, 86, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 74.3
Expected points: 52.524057388305664


,element_type,web_name,now_cost,games played,gameweek 30 prediction
id,,,,,
475,1,Sá,53,29.0,4.287109
16,2,Tierney,51,21.0,3.886938
360,2,Doherty,48,12.0,4.032229
67,2,White,45,25.0,3.738794
196,3,Raphinha,65,27.0,4.745955
22,3,Saka,66,27.0,4.184176
359,3,Son,109,25.0,5.147920
681,3,Coutinho,75,9.0,4.422555
701,3,Kulusevski,62,8.0,4.676741


In [8]:
df = df.drop([205])
optimize(df, 86, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 71.69999999999999
Expected points: 52.49398851394653


,element_type,web_name,now_cost,games played,gameweek 30 prediction
id,,,,,
475,1,Sá,53,29.0,4.287109
16,2,Tierney,51,21.0,3.886938
360,2,Doherty,48,12.0,4.032229
67,2,White,45,25.0,3.738794
196,3,Raphinha,65,27.0,4.745955
22,3,Saka,66,27.0,4.184176
359,3,Son,109,25.0,5.147920
681,3,Coutinho,75,9.0,4.422555
701,3,Kulusevski,62,8.0,4.676741


In [9]:
df = df.drop([189])
optimize(df, 86, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 71.6
Expected points: 52.451236724853516


,element_type,web_name,now_cost,games played,gameweek 30 prediction
id,,,,,
475,1,Sá,53,29.0,4.287109
16,2,Tierney,51,21.0,3.886938
360,2,Doherty,48,12.0,4.032229
67,2,White,45,25.0,3.738794
196,3,Raphinha,65,27.0,4.745955
22,3,Saka,66,27.0,4.184176
359,3,Son,109,25.0,5.147920
681,3,Coutinho,75,9.0,4.422555
701,3,Kulusevski,62,8.0,4.676741


In [10]:
optimize(df, 90, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 71.6
Expected points: 52.451236724853516


,element_type,web_name,now_cost,games played,gameweek 30 prediction
id,,,,,
475,1,Sá,53,29.0,4.287109
16,2,Tierney,51,21.0,3.886938
360,2,Doherty,48,12.0,4.032229
67,2,White,45,25.0,3.738794
196,3,Raphinha,65,27.0,4.745955
22,3,Saka,66,27.0,4.184176
359,3,Son,109,25.0,5.147920
681,3,Coutinho,75,9.0,4.422555
701,3,Kulusevski,62,8.0,4.676741


In [11]:
df = df.drop([360])
optimize(df, 90, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 75.99999999999999
Expected points: 52.38368320465088


,element_type,web_name,now_cost,games played,gameweek 30 prediction
id,,,,,
475,1,Sá,53,29.0,4.287109
16,2,Tierney,51,21.0,3.886938
590,2,Tomiyasu,46,16.0,3.682068
67,2,White,45,25.0,3.738794
196,3,Raphinha,65,27.0,4.745955
210,3,Tielemans,64,21.0,4.183875
359,3,Son,109,25.0,5.147920
681,3,Coutinho,75,9.0,4.422555
701,3,Kulusevski,62,8.0,4.676741
